In [4]:
import numpy as np
import pandas as pd
import random as rd
from collections import defaultdict
%pylab inline

Populating the interactive namespace from numpy and matplotlib


### Load Data 

In [2]:
train = pd.read_csv('train.csv')
train = train.set_index('ID')

/Users/zach/anaconda/lib/python2.7/site-packages/pandas/io/parsers.py:1170: DtypeWarning: Columns (8,9,10,11,12,43,157,196,214,225,228,229,231,235,238) have mixed types. Specify dtype option on import or set low_memory=False.
  data = self._reader.read(nrows)


Get a sample to play with if we need it

In [6]:
samp = train.loc[rd.sample(train.index, 5000)]

### EDA

In [7]:
# Variable types
vartypes = train.dtypes
vartypes

cat_df = train.loc[:, vartypes[vartypes=='O'].index]
int_df = train.loc[:, vartypes[vartypes=='int64'].index]
float_df = train.loc[:, vartypes[vartypes=='float64'].index]


In [8]:
# Variable types
vartypes = train.dtypes
vartypes.value_counts()



int64      1406
float64     477
object       51
dtype: int64

#### Categorical

The first thing we need to do is make a summary of all the categorical variables, so that we can figure out how to process them. Things I would like to look at:
- Number of unique values
- Frequency of top value
- Number of missing values
- First few examples

In [18]:
pd.Series.value_counts?

In [19]:
def get_summary(col):
    cnts = col.value_counts(normalize=True)
    num_unique = len(cnts)
    top_freq = cnts[0]
    top_item = cnts.index[0]
    fraction_missing = col.isnull().mean()
    first_few = ', '.join(list(col.unique()
                               .astype(str)[:max(num_unique, 3)]))
    return pd.Series(
        {'NumUnique' : num_unique,
         'Top Freq' : top_freq,
         'Top Item' : top_item,
         'Fraction Missing' : fraction_missing,
         'Examples' : first_few}
        )

cat_df.apply(get_summary).T

,Examples,Fraction Missing,NumUnique,Top Freq,Top Item
VAR_0001,"H, R, Q",0,3,0.5845377,R
VAR_0005,"C, B, N, S",0,4,0.491968,B
VAR_0008,"False, nan",0.0003855926,1,0.9996144,False
VAR_0009,"False, nan",0.0003855926,1,0.9996144,False
VAR_0010,"False, nan",0.0003855926,1,0.9996144,False
VAR_0011,"False, nan",0.0003855926,1,0.9996144,False
VAR_0012,"False, nan",0.0003855926,1,0.9996144,False
VAR_0043,"False, nan",0.0003855926,1,0.9996144,False
VAR_0044,"[], nan",0.0003855926,1,0.9996144,[]
VAR_0073,"nan, 04SEP12:00:00:00, 26JAN12:00:00:00, 18SEP...",0.6963183,1458,0.001790251,13MAR09:00:00:00


In [108]:
var = 'VAR_0237' 
grouped = train.groupby(var)['target'].aggregate({'Response Rate' : mean, 'Total' : len})
grouped

,Response Rate,Total
VAR_0237,,
AK,0.000000,1
AL,0.292477,4985
AR,0.000000,9
AZ,0.232029,1099
CA,0.228916,15770
CO,0.217949,2808
CT,1.000000,1
DC,0.214286,28
DE,0.186335,161


In [103]:
num_values = cat_df.apply(lambda x: x.nunique())
num_values

VAR_0001       3
VAR_0005       4
VAR_0008       1
VAR_0009       1
VAR_0010       1
VAR_0011       1
VAR_0012       1
VAR_0043       1
VAR_0044       1
VAR_0073     680
VAR_0075    1112
VAR_0156     172
VAR_0157      28
VAR_0158      64
VAR_0159     167
VAR_0166     404
VAR_0167      84
VAR_0168     277
VAR_0169     383
VAR_0176     480
VAR_0177     107
VAR_0178     301
VAR_0179     415
VAR_0196       1
VAR_0200    2280
VAR_0202       1
VAR_0204    1158
VAR_0214       0
VAR_0216       1
VAR_0217     390
VAR_0222       1
VAR_0226       2
VAR_0229       1
VAR_0230       2
VAR_0232       2
VAR_0236       2
VAR_0237      42
VAR_0239       1
VAR_0274      56
VAR_0283       6
VAR_0305       7
VAR_0325       8
VAR_0342      46
VAR_0352       4
VAR_0353       4
VAR_0354       4
VAR_0404     129
VAR_0466       2
VAR_0467       4
VAR_0493     108
VAR_1934       5
dtype: int64

#### Integer
 
- Each integer valued variable needs to be checked to see if it should be treated as categorical or numerical
- It's possible these can all be taken care of, just by using trees

#### Are all the missing values (99, 999, etc..) in the same spot?

In [90]:
int_df.apply(lambda x: x.nunique())
# int_df['VAR_0532'].value_counts()

VAR_0002     475
VAR_0003     381
VAR_0004    1956
VAR_0532       7
VAR_0533       7
VAR_0534      10
VAR_0535      19
VAR_0536      31
VAR_0537      13
VAR_0538       7
VAR_0539      13
VAR_0540      86
VAR_0541    4538
VAR_0542    1212
VAR_0543    4396
VAR_0544    1897
VAR_0545       7
VAR_0546       8
VAR_0547       8
VAR_0548       8
VAR_0549       8
VAR_0550     145
VAR_0551       8
VAR_0552      39
VAR_0553      32
VAR_0554     149
VAR_0555      86
VAR_0556      85
VAR_0557      84
VAR_0558      85
            ... 
VAR_1905       5
VAR_1906      47
VAR_1907      24
VAR_1908      21
VAR_1909      24
VAR_1910      21
VAR_1911      39
VAR_1912     729
VAR_1913    1209
VAR_1914     665
VAR_1915     335
VAR_1916       8
VAR_1917      10
VAR_1918      81
VAR_1919     235
VAR_1920      14
VAR_1921      12
VAR_1922     435
VAR_1923     244
VAR_1924     322
VAR_1925       7
VAR_1926       4
VAR_1927       6
VAR_1928      15
VAR_1929      37
VAR_1930       7
VAR_1931       9
VAR_1932      

In [92]:
train[col].value_counts()

0     79724
1     29028
2     15340
3      8368
4      4925
5      2775
6      1697
7       973
8       631
99      601
9       399
10      214
11      161
12       99
13       78
14       69
15       49
16       32
17       17
18       14
19       11
20        6
24        5
23        3
21        3
25        2
28        2
34        1
22        1
26        1
27        1
30        1
dtype: int64

In [98]:
test.shape

(145232, 1933)

In [95]:
col = 'VAR_0537'
grouped = train.groupby(col)['target'].aggregate({'mean' : mean, 'count' : len})
grouped

,count,mean
VAR_0537,,
0,89230,0.248605
1,34649,0.201305
2,12669,0.202857
3,4759,0.212650
4,1846,0.224269
5,734,0.235695
6,329,0.231003
7,159,0.182390
8,99,0.202020


#### Float
- Float variables _should_ be the easiest---they can be taken at face value

In [67]:

float_df.apply(lambda x: x.nunique())

VAR_0006     19
VAR_0007     16
VAR_0013     19
VAR_0014     15
VAR_0015     12
VAR_0016     13
VAR_0017     12
VAR_0018      1
VAR_0019      1
VAR_0020      1
VAR_0021      1
VAR_0022      1
VAR_0023      1
VAR_0024      1
VAR_0025      1
VAR_0026      1
VAR_0027      1
VAR_0028      1
VAR_0029      1
VAR_0030      1
VAR_0031      1
VAR_0032      1
VAR_0033      9
VAR_0034     15
VAR_0035     15
VAR_0036     10
VAR_0037      9
VAR_0038      1
VAR_0039      1
VAR_0040      1
           ... 
VAR_0503      2
VAR_0504      2
VAR_0505      2
VAR_0506     10
VAR_0507      2
VAR_0508      2
VAR_0509      5
VAR_0510      9
VAR_0511     10
VAR_0512     10
VAR_0513      3
VAR_0514    154
VAR_0515     95
VAR_0516    163
VAR_0517    109
VAR_0518      3
VAR_0519      6
VAR_0520      3
VAR_0521      2
VAR_0522      2
VAR_0523      2
VAR_0524      3
VAR_0525      4
VAR_0526      1
VAR_0527      1
VAR_0528      1
VAR_0529      1
VAR_0530      1
VAR_0531     13
VAR_0840      0
dtype: int64

In [79]:
float_df.iloc[2].value_counts()

 0              306
 1               72
-1               28
 3                9
 2                7
 189              4
 42               3
 2011             3
 37660            3
 4                3
 15               3
 190              2
 68140            2
 52               2
 32490            2
 478870           2
 6                1
 7                1
 201              1
 266              1
 11               1
 37               1
 9                1
 79703            1
 79703681309      1
 290              1
 101              1
 2471             1
 201111           1
-35650            1
 35000            1
 217              1
dtype: int64

In [80]:
float_df.columns[2]

'VAR_0013'

Combine models from all three variable types

In [83]:
int_df.

0       113
2000    100
3000     98
2500     88
4000     81
2400     66
1800     63
1600     61
2200     59
3500     55
1500     55
2800     55
1200     53
1400     52
2600     51
2300     47
5000     45
4500     40
1700     40
3200     37
2604     35
1000     34
6000     31
1900     31
2387     30
4200     30
3255     29
2100     26
2821     26
3600     25
       ... 
3099      1
3103      1
3111      1
1078      1
5064      1
3007      1
822       1
5048      1
4928      1
3546      1
4936      1
2899      1
2903      1
2907      1
866       1
870       1
4968      1
2927      1
2935      1
902       1
2951      1
2959      1
5012      1
926       1
938       1
2987      1
9134      1
2991      1
5040      1
3996      1
dtype: int64